In [148]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import sys
from numpy import *
from numpy.linalg import *


eps=sys.float_info.epsilon

#EG con pivot

def pivot(a:np.array,i)->int:
    max=abs(a[i,i])
    res=0
    for k in range(i+1,len(a)):
        if(abs(a[k,i])>max):
            max=a[k,i]
            res=k
    return res

def producto_punto(v:np.array,w:np.array)->np.float64:
    res=0
    for i in range(0,v.size):
        res+=v[i]*w[i]
    return res

#Para poder limitar el error numérico y que el algoritmo sea correcto (i.e. devuelva S.I. cuando no hay solucion o alguna si existe) , consideramos que un x<=e-5 es efectivamente 0.Es decir , nos basamos en una precision de 5 digitos en los resultados.

def eliminacion_gaussiana_pivot(m:np.array,b:np.array)->np.array:
    if(len(m)!=len(m[0])):
        print("Error. Matriz no cuadrada")

    #Construimos A' , extension de A con b
    n=len(m)
    m_ext=np.hstack((m,np.zeros((n,1))))
    m_ext[:,n]=b


    for i in range(0,n):

        fila_a_permutar=pivot(m_ext,i)
        if(fila_a_permutar>i):      #Para el caso que devuelve 0
            copy=np.array(m_ext[i,:])
            m_ext[i,:]=m_ext[fila_a_permutar,:]
            m_ext[fila_a_permutar,:]=copy

        #Si la columna ya es nula,paso.
        if(np.all(m_ext[:,i+1:] == 0)):
            continue

        for j in range(i+1,n):
            if(np.abs(m_ext[i,i])<eps):
                print("Posible Error Numérico.Cociente con divisor cercano a 0.\n")
            multiplier=m_ext[j,i]/m_ext[i,i]
            m_ext[j,:]=m_ext[j,:]-m_ext[i,:]*multiplier     #M_j=M_j - M_i * multiplier

    #Excepcion a devolver
    cota_decimal=eps*pow(10,6)
    for i in range(0,n):
        if(np.abs(m_ext[i,i])>cota_decimal): #M_ii==0        (A CHEQUEAR)
            continue
        elif(np.abs(m_ext[i,n])>cota_decimal):#    b_ii==0                    (A CHEQUEAR)
            print("Sistema incompatible . No existen soluciones.")
            return 
        else:
            print("Hay infinitas soluciones.")            


    #Ya A' escalonada -> despejamos x
    x=np.zeros(n)
    for i in range(n-1,-1,-1):
        if(np.abs(m_ext[i,i])<eps):
                print("Posible Error Numérico.Cociente con divisor cercano a 0.\n")
        b_i=m_ext[i,n]
        if(i==n-1):
            x[i]=b_i/m_ext[i,i]
            continue
        prod=producto_punto(m_ext[i,i+1:n],x[i+1:n])
        x[i]=(b_i-prod)/m_ext[i,i]

    #Devolvemos X=[x1,...,xn]
    return x




#matrix=np.array([[0,1,1,2],
#                 [2,-1,1,0],
#                 [3,1,0,1],
#                 [0,2,1,3]],dtype=np.float64)
#
#b=np.array([1,2,1,0],dtype=np.float64)

matrix=np.array([[3,1,2],
                [3,0,1],
                [1,0,2]],dtype=np.float32)

b=np.array([1,1,1],dtype=np.float32)

# AX0=?b
print(matrix@eliminacion_gaussiana_pivot(matrix,b))


solve(matrix,b)

[[ 3.          1.          2.          1.        ]
 [ 0.         -1.         -1.          0.        ]
 [ 0.          0.          1.66666667  0.66666667]]
[1.         0.         0.66666667]
[ 0.2 -0.4  0.4]
[1. 1. 1.]


array([ 0.2, -0.4,  0.4], dtype=float32)

np.float64(144717.8002894356)